In [10]:
import pandas as pd
import importlib  

nb = importlib.import_module("ipynb.fs.defs.battery-stats")

In [11]:
get_rows_before_charging = nb.get_rows_before_charging

In [12]:
def build_battery_history(values, device_id="60df87b4be3fc900069dac1f", start_time=1600000000000):
    id_array = [device_id] * len(values)
    timestamp_array = map(lambda t: start_time + t * 10000, range(len(values)))
    return pd.DataFrame({
        "ID": pd.Series(id_array),
        "battery": pd.Series(values, dtype="int64"),
        "Timestamp": pd.Series(timestamp_array, dtype="int64")
    }).set_index('Timestamp')

In [13]:
# it should return an empty DataFrame if the battery was never recharged
df_descending_only = build_battery_history([100,99,50,0])
assert len( get_rows_before_charging(df_descending_only) ) == 0

In [14]:
# it should return an empty DataFrame if the battery was already charging and charges the whole time
df_ascending_only = build_battery_history([0,50,99,100])
assert len( get_rows_before_charging(df_ascending_only) ) == 0

In [15]:
# it should return only the row(s) immediately before the battery was charging
df_single_ascent = build_battery_history([100,50,60,40])
result_single_ascent = get_rows_before_charging(df_single_ascent)
assert len(result_single_ascent) == 1
assert result_single_ascent.iloc[0]['battery'] == 50

In [16]:
# it should return multiple rows if there are multiple charge cycles
df_two_ascents = build_battery_history([100,50,60,40,80])
result_two_ascents = get_rows_before_charging(df_two_ascents)
assert len(result_two_ascents) == 2
assert result_two_ascents['battery'].mean() == 45

In [17]:
# it should consider each device of multiple devices separately
df_two_devices_a = build_battery_history([100,50,90,20])
df_two_devices_b = build_battery_history([100,0,90],device_id="79df87b4be3fc900069dbc33") # going from 20 to 100 should *not* be a charge
df_two_devices = pd.concat([df_two_devices_a, df_two_devices_b])
result_two_devices = get_rows_before_charging(df_two_devices)
assert len(result_two_devices) == 2
assert result_two_devices['battery'].mean() == 25     

In [20]:
# it should ignore rows with the same value as the previous row when determining change
df_duplicate_values = build_battery_history([100,20,20,50])
assert len( get_rows_before_charging(df_duplicate_values) ) == 1